In [1]:
import numpy as np
import pandas as pd
from mp_api.client import MPRester
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from matminer.featurizers.structure import DensityFeatures
from matminer.featurizers.structure import StructuralComplexity
from matminer.featurizers.structure import MaximumPackingEfficiency

C:\Users\Alexander\anaconda3\envs\materials\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Perovskites

In [2]:
with MPRester("EclqWMXn0DY3CAOSELD3xvCjEVhlcEYp") as mpr:
    perovskites = mpr.summary.search(formula=["ABC3"], fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9340\4088062957.py:2: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  perovskites = mpr.summary.search(formula=["ABC3"], fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])
Retrieving SummaryDoc documents: 100%|██████████| 4555/4555 [00:07<00:00, 623.81it/s]


In [5]:
ids = []
for i in range(len(perovskites)):
    single = perovskites[i]
    ids.append(single.material_id)

structures = []
for i in range(len(perovskites)):
    single = perovskites[i]
    structures.append(single.structure)

band_gaps = []
for i in range(len(perovskites)):
    single = perovskites[i]
    band_gaps.append(single.band_gap)

theory = []
for i in range(len(perovskites)):
    single = perovskites[i]
    theory.append(single.theoretical)

stable = []
for i in range(len(perovskites)):
    single = perovskites[i]
    stable.append(single.is_stable)

formula = []
for i in range(len(perovskites)):
    single = perovskites[i]
    formula.append(single.formula_pretty)

composition = []
for i in range(len(perovskites)):
    single = perovskites[i]
    composition.append(single.composition)

perov_df = pd.DataFrame(
    {"material ids": ids,
     "formula": formula,
     "composition": composition,
     "structure": structures,
     "band gaps": band_gaps,
     "theoretical": theory,
     "stable": stable
    })

In [7]:
real_perovs = perov_df[perov_df["theoretical"]==False]

In [8]:
real_perovs["stable"] = real_perovs["stable"].astype(int)

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9340\3917787968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_perovs["stable"] = real_perovs["stable"].astype(int)


In [9]:
perovs_feat = real_perovs

In [11]:
densityf = DensityFeatures()
strcomp = StructuralComplexity()
mpe = MaximumPackingEfficiency()

In [12]:
perovs_feat = densityf.featurize_dataframe(perovs_feat, "structure")

DensityFeatures: 100%|██████████| 1759/1759 [04:42<00:00,  6.22it/s]


In [13]:
perovs_feat = strcomp.featurize_dataframe(perovs_feat, "structure")

StructuralComplexity: 100%|██████████| 1759/1759 [04:49<00:00,  6.08it/s]


In [14]:
perovs_feat = mpe.featurize_dataframe(perovs_feat, "structure")

MaximumPackingEfficiency: 100%|██████████| 1759/1759 [11:45<00:00,  2.49it/s]


In [15]:
y = perovs_feat["stable"].values
X = perovs_feat.drop(["material ids", "formula", "composition", "structure", "band gaps", "theoretical", "stable"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

stable_classifier = RandomForestClassifier(random_state=0)
stable_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [16]:
training_accuracy = stable_classifier.score(X_train, y_train) * 100
print(f'Accuracy on training data: {training_accuracy:.1f} %')
testing_accuracy = stable_classifier.score(X_test, y_test) * 100
print(f'Accuracy on testing data: {testing_accuracy:.1f} %')

Accuracy on training data: 100.0 %
Accuracy on testing data: 71.1 %


### III-V Semiconductors

In [17]:
with MPRester("EclqWMXn0DY3CAOSELD3xvCjEVhlcEYp") as mpr:
    iii_v = mpr.summary.search(chemsys=["Al-N", "Al-P", "Al-As", "Al-Sb",
                                      "Ga-N", "Ga-P", "Ga-As", "Ga-Sb"
                                      "In-N", "In-P", "In-As", "In-Sb"], 
                              fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9340\2391299554.py:2: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  iii_v = mpr.summary.search(chemsys=["Al-N", "Al-P", "Al-As", "Al-Sb",
Retrieving SummaryDoc documents: 100%|██████████| 117/117 [00:00<?, ?it/s]


In [19]:
ids = []
for i in range(len(iii_v)):
    single = iii_v[i]
    ids.append(single.material_id)

structures = []
for i in range(len(iii_v)):
    single = iii_v[i]
    structures.append(single.structure)

band_gaps = []
for i in range(len(iii_v)):
    single = iii_v[i]
    band_gaps.append(single.band_gap)

theory = []
for i in range(len(iii_v)):
    single = iii_v[i]
    theory.append(single.theoretical)

stable = []
for i in range(len(iii_v)):
    single = iii_v[i]
    stable.append(single.is_stable)

formula = []
for i in range(len(iii_v)):
    single = iii_v[i]
    formula.append(single.formula_pretty)

composition = []
for i in range(len(iii_v)):
    single = iii_v[i]
    composition.append(single.composition)

iii_v_df = pd.DataFrame(
    {"material ids": ids,
     "formula": formula,
     "composition": composition,
     "structure": structures,
     "band gaps": band_gaps,
     "theoretical": theory,
     "stable": stable
    })

In [20]:
iii_v_feat = iii_v_df

In [21]:
iii_v_feat = densityf.featurize_dataframe(iii_v_feat, "structure")

DensityFeatures: 100%|██████████| 117/117 [00:10<00:00, 10.75it/s]


In [22]:
iii_v_feat = strcomp.featurize_dataframe(iii_v_feat, "structure")

StructuralComplexity: 100%|██████████| 117/117 [00:13<00:00,  8.51it/s]


In [23]:
iii_v_feat = mpe.featurize_dataframe(iii_v_feat, "structure")

MaximumPackingEfficiency: 100%|██████████| 117/117 [00:16<00:00,  7.14it/s]


In [24]:
y = iii_v_feat["stable"].values
X = iii_v_feat.drop(["material ids", "formula", "composition", "structure", "band gaps", "theoretical", "stable"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

stable_classifier = RandomForestClassifier(random_state=0)
stable_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [25]:
training_accuracy = stable_classifier.score(X_train, y_train) * 100
print(f'Accuracy on training data: {training_accuracy:.1f} %')
testing_accuracy = stable_classifier.score(X_test, y_test) * 100
print(f'Accuracy on testing data: {testing_accuracy:.1f} %')

Accuracy on training data: 100.0 %
Accuracy on testing data: 96.7 %


### II-VI Semiconductors

In [26]:
with MPRester("EclqWMXn0DY3CAOSELD3xvCjEVhlcEYp") as mpr:
    ii_vi = mpr.summary.search(chemsys=["Zn-S", "Zn-Se", "Zn-Te",
                                        "Cd-S", "Cd-Se", "Cd-Te"], 
                               fields = ["material_id", "structure", "band_gap", "theoretical", "is_stable", "formula_pretty", "composition"])

C:\Users\Alexander\AppData\Local\Temp\ipykernel_9340\1361324371.py:2: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  ii_vi = mpr.summary.search(chemsys=["Zn-S", "Zn-Se", "Zn-Te",
Retrieving SummaryDoc documents: 100%|██████████| 185/185 [00:00<?, ?it/s]


In [27]:
ids = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    ids.append(single.material_id)

structures = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    structures.append(single.structure)

band_gaps = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    band_gaps.append(single.band_gap)

theory = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    theory.append(single.theoretical)

stable = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    stable.append(single.is_stable)

formula = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    formula.append(single.formula_pretty)

composition = []
for i in range(len(ii_vi)):
    single = ii_vi[i]
    composition.append(single.composition)

ii_vi_df = pd.DataFrame(
    {"material ids": ids,
     "formula": formula,
     "composition": composition,
     "structure": structures,
     "band gaps": band_gaps,
     "theoretical": theory,
     "stable": stable
    })

In [28]:
ii_vi_feat = ii_vi_df

In [29]:
ii_vi_feat = densityf.featurize_dataframe(ii_vi_feat, "structure")

DensityFeatures: 100%|██████████| 185/185 [00:24<00:00,  7.55it/s]


In [30]:
ii_vi_feat = strcomp.featurize_dataframe(ii_vi_feat, "structure")

StructuralComplexity: 100%|██████████| 185/185 [00:26<00:00,  7.02it/s]


In [31]:
ii_vi_feat = mpe.featurize_dataframe(ii_vi_feat, "structure")

MaximumPackingEfficiency: 100%|██████████| 185/185 [01:17<00:00,  2.39it/s]


In [32]:
y = ii_vi_feat["stable"].values
X = ii_vi_feat.drop(["material ids", "formula", "composition", "structure", "band gaps", "theoretical", "stable"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

stable_classifier = RandomForestClassifier(random_state=0)
stable_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [33]:
training_accuracy = stable_classifier.score(X_train, y_train) * 100
print(f'Accuracy on training data: {training_accuracy:.1f} %')
testing_accuracy = stable_classifier.score(X_test, y_test) * 100
print(f'Accuracy on testing data: {testing_accuracy:.1f} %')

Accuracy on training data: 100.0 %
Accuracy on testing data: 95.7 %
